In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pickle
import pmdarima as pm  # Import Auto ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

df = pd.read_csv('C:\\Users\\dhruv\\Downloads\\prepared_time_series_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df.set_index('timestamp', inplace=True)

df.dropna(inplace=True)

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)  # 80% for training
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Fit the model using Auto ARIMA on training data
cpu_model_fit = pm.auto_arima(train['cpu_utilization'], seasonal=False, stepwise=True)


n_periods = len(test)
future_forecast, conf_int = cpu_model_fit.predict(n_periods=n_periods, return_conf_int=True)

mae = mean_absolute_error(test['cpu_utilization'], future_forecast)
mse = mean_squared_error(test['cpu_utilization'], future_forecast)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


window = 50  
smoothed_actual = test['cpu_utilization'].rolling(window).mean()
smoothed_predicted = pd.Series(future_forecast, index=test.index).rolling(window).mean()

plt.figure(figsize=(12, 6))


plt.scatter(test.index, test['cpu_utilization'], label="Actual CPU Usage", color='blue', alpha=0.3, s=10)


plt.plot(test.index, smoothed_predicted, label="Predicted CPU Usage", color='orange', linestyle="dashed", linewidth=2)


accuracy_text = f"MAE: {mae:.4f}\nMSE: {mse:.4f}\nRMSE: {rmse:.4f}"
plt.gca().text(0.01, 0.95, accuracy_text, transform=plt.gca().transAxes,
               fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))


plt.xlabel("Time")
plt.ylabel("CPU Usage")
plt.title("Actual vs Predicted CPU Usage (Improved Visualization)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
